In [32]:
import json
import numpy as np
from scipy.spatial.distance import cdist, euclidean
import pickle
import os
import yaml

In [53]:
data_dir = 'D:\\Dropbox\\research\\#MARS_AND_BENTO\\AMT_reannotation\\datasets\\white_100\\'
manifest_name = 'white_mouse_100_output'
nWorkers = 6
config = 'project_config.yaml'

In [101]:
# first read the config and the output manifest:
fid = data_dir + manifest_name + ('.manifest' if not manifest_name.endswith('.manifest') else '')

data = []
for line in open(fid,'r'):
    data.append(json.loads(line))

with open(config) as f:
    cfg = yaml.load(f)

keypoints = cfg['keypoints']
animal_names = cfg['animal_names'] if cfg['animal_names'] else animal
species = cfg['species']

FileNotFoundError: [Errno 2] No such file or directory: 'annot_config.yml'

In [127]:
animal_names=['mouse']

In [133]:
nWorkers=10

In [148]:
np.median(rawPts['mouse'],axis=2)

array([[ 685.5,  543. ],
       [ 755.5,  419. ],
       [ 753.5,  434. ],
       [ 792.5,  409. ],
       [ 920. ,  512.5],
       [ 925. ,  510. ],
       [1007. ,  490.5]])

In [144]:
# next we unpack the raw data from all individual annotators

nKpts   = len(keypoints)

for f, sample in enumerate(data):
    if ('annotatedResult' in sample.keys()): #check if this frame has at least one set of annotations

        rawPts = {n:np.zeros((nKpts, 2, nWorkers)) for n in animal_names}
        for w,worker in enumerate(sample['annotatedResult']['annotationsFromAllWorkers']):
            workerCount[f] = workerCount[f] + 1

            # the json of annotations from each worker is stored as a string for security reasons.
            # we'll use eval to convert it into a dict:
            annot = eval(worker['annotationData']['content'])

            # now we can unpack this worker's annotations for each keypoint:
            for pt in annot['annotatedResult']['keypoints']:
                animal = next((n for n in animal_names if n in pt['label']),cfg['species'])
                part = keypoints.index(pt['label'].replace(animal, '').replace(species, '').strip())

                rawPts[animal][part,0,w] = pt['x'] #/im.shape[1]
                rawPts[animal][part,1,w] = pt['y'] #/im.shape[0]
        

In [103]:
# next we unpack the raw data from all individual annotators

nSamp   = len(data)
nKpts   = len(keypoints)

# for each image: nKpts keypoints, 2 dimensions, nWorkers workers per keypoint
rawPts={}

rawPts = {n:np.zeros((nSamp,nKpts,2,nWorkers)) for n in animal_names}

images = ['']*nSamp        # track paths to labeled images
hits = [False]*nSamp       # track which images have annotations
workerCount = [0]*nSamp    # track the number of workers who labeled each image

sourceStr = os.path.dirname(data[0]['source-ref']) # name of the bucket images are in on AWS

for f,frame in enumerate(data):
    if ('annotatedResult' in frame.keys()): #check if this frame has at least one set of annotations
        hits[f] = True

        images[f] = frame['source-ref']

        for w,worker in enumerate(frame['annotatedResult']['annotationsFromAllWorkers']):
            workerCount[f] = workerCount[f] + 1

            # the json of annotations from each worker is stored as a string for security reasons.
            # we'll use eval to convert it into a dict:
            annot = eval(worker['annotationData']['content'])

            # now we can unpack this worker's annotations for each keypoint:
            for pt in annot['annotatedResult']['keypoints']:
                animal = next((n for n in animal_names if n in pt['label']),'animal')
                idx = keypoints.index(pt['label'].replace(animal,'').replace(species,'').strip())
                
                rawPts[animal][f,idx,0,w] = pt['x']
                rawPts[animal][f,idx,1,w] = pt['y']


TypeError: string indices must be integers

In [97]:
print(frame)
print(frame2)

[[[ 704.  682.  684.  687.  693.  683.]
  [ 534.  547.  543.  543.  541.  544.]]

 [[ 786.  797.  725.  712.  782.  729.]
  [ 452.  435.  422.  399.  444.  416.]]

 [[ 723.  715.  784.  804.  722.  815.]
  [ 435.  415.  451.  408.  433.  426.]]

 [[ 806.  792.  803.  780.  793.  782.]
  [ 437.  418.  410.  408.  405.  408.]]

 [[ 871.  908.  952.  932.  937.  874.]
  [ 521.  473.  543.  529.  504.  483.]]

 [[ 920.  930.  910.  937.  849.  957.]
  [ 444.  542.  511.  580.  507.  509.]]

 [[ 977.  988.  999. 1016. 1023. 1015.]
  [ 496.  485.  483.  522.  461.  534.]]]
[[[ 704.  682.  684.  687.  693.  683.]
  [ 534.  547.  543.  543.  541.  544.]]

 [[ 786.  797.  725.  804.  782.  729.]
  [ 452.  435.  422.  408.  444.  416.]]

 [[ 723.  715.  784.  712.  722.  815.]
  [ 435.  415.  451.  399.  433.  426.]]

 [[ 806.  792.  803.  780.  793.  782.]
  [ 437.  418.  410.  408.  405.  408.]]

 [[ 871.  908.  952.  932.  937.  874.]
  [ 521.  473.  543.  529.  504.  483.]]

 [[ 920.  930.  

In [96]:
# now let's get our ground-truth keypoints by taking the median across annotators.

gtPts  = {n:[] for n in animal_names}
allPts = {n:[] for n in animal_names}

for animal in animal_names:
    #first we'll take medians of the raw keypoints:
    gtPts[animal] = np.zeros((nSamp,nKpts,2))
    f=0
    rame = rawPts[animal][0]
#     for f,frame in enumerate(rawPts[animal]):
    meds = np.median(np.squeeze(frame.T),axis=0)

    gtPts[animal][f,:,:] = meds.T
    frame2 = np.copy(frame)
    i1 = 1
    i2 = 2
    frame[[i1, i2],:,3] = frame[[i2, i1],:,3]

    for w,worker in enumerate(frame.swapaxes(0,2).swapaxes(1,2)):
            d1 = cdist(worker[[i1,i2],:],[meds[:,i1]])
            d2 = cdist(worker[[i1,i2],:],[meds[:,i2]])
            if (d1[0]>d2[1]) and (d2[1]>d1[0]):
                frame2[[i1, i2],:,w] = frame[[i2, i1],:,w]

    # then we'll adjust L/R assignments to try to find better median estimates. Iterating
    # seems to help settle on better final keypoints if the initial guesses were especially bad.
#     for rep in range(1):
#         # for ears and hips we should check for + correct Left/Right swaps, a common annotator mistake:
#         allPts[animal] = np.copy(rawPts[animal])
        
#         for fr,(gt,frame) in enumerate(zip(gtPts[animal],rawPts[animal])):
#             for w,worker in enumerate(frame.swapaxes(0,2).swapaxes(1,2)):
                
#                 dEar1 = cdist(worker[1:3,:],[gt[1]])
#                 dEar2 = cdist(worker[1:3,:],[gt[2]])
#                 if (dEar1[0]>dEar1[1]) and (dEar2[1]>dEar1[0]):
#                     allPts[animal][fr,1,:,w] = rawPts[animal][fr,2,:,w]
#                     allPts[animal][fr,2,:,w] = rawPts[animal][fr,1,:,w]
                
#                 dHip1 = cdist(worker[4:6,:],[gt[4]])
#                 dHip2 = cdist(worker[4:6,:],[gt[5]])
#                 if (dHip1[0]>dHip1[1]) and (dHip2[1]>dHip2[0]):
#                     allPts[animal][fr,4,:,w] = rawPts[animal][fr,5,:,w]
#                     allPts[animal][fr,5,:,w] = rawPts[animal][fr,4,:,w]

#         # then re-compute the medians:
#         gtPts[animal] = np.zeros((nSamp,nKpts,2))
#         for f,frame in enumerate(allPts[animal]):
#             for p,pt in enumerate(frame):
#                 gtPts[animal][f,p,:] = geometric_median(np.squeeze(pt.T))

In [ ]:
# let's look at a couple example images to see how our annotators did.
# (note, this also saves the figure as a pdf in lbl_bucket):

# each body part gets an assigned color; each annotator gets an assigned marker.
colors  = 'rgbcmyw'
markers = 'v+xosd'

fig,ax = plt.subplots(5,2,figsize=(16, 30))
for fr in range(0,10):
    im = mpimg.imread(images[fr])
    ax[int(fr/2),(fr+1)%2].imshow(im);
    
    # plot the labels from each individual worker:
    for mouse,mouseColor in zip(['white','black'],['w','k']):
        for p,pt in enumerate(np.squeeze(allPts[mouse][fr,...])):
            for w in range(nWorkers):
                ax[int(fr/2),(fr+1)%2].plot(pt[0,w], pt[1,w],colors[p] + markers[w], markersize=2,markeredgecolor=mouseColor)

        # plot the "ground truth" labels:
        ax[int(fr/2),(fr+1)%2].plot(gtPts[mouse][fr,:,0], gtPts[mouse][fr,:,1],'k.', markersize=4)

fig.savefig('s3://{}/sample_annotations.pdf'.format(lbl_bucket), bbox_inches='tight')

In [ ]:
# now we'd like to quantify annotator variability. For some reason, this seems to be a good predictor
# of how well our pose estimator is going to do once trained.

pixels_per_cm = 37.795

fig,ax = plt.subplots(2,4,figsize=(16, 8))
dashes = {'white':'dotted', 'black':'dashed', 'both':'solid'}
colors = 'rbg'

for mouse in ['white','black','both']:
    if mouse != 'both':
        dMean = np.zeros((nKpts,nSamp)) # average worker-gt distance
        dMin  = np.zeros((nKpts,nSamp)) # performance of best worker on a given frame
        dMax  = np.zeros((nKpts,nSamp)) # performance of worst worker on a given frame
        for fr,(gt,est) in enumerate(zip(gtPts[mouse],allPts[mouse])):
            for pt,(gtPt,estPts) in enumerate(zip(gt,est)):
                D = cdist(estPts.T, [gtPt]) / pixels_per_cm
                dMean[pt,fr] = np.mean(D)
                dMin[pt,fr]  = min(D)
                dMax[pt,fr]  = max(D)
    else:
        dMean = np.zeros((nKpts,nSamp*2))
        dMin  = np.zeros((nKpts,nSamp*2))
        dMax  = np.zeros((nKpts,nSamp*2))
        for fr,(gt,est) in enumerate(zip(np.concatenate((gtPts['white'],gtPts['black'])), \
                                        np.concatenate((allPts['white'],allPts['black'])))):
            for pt,(gtPt,estPts) in enumerate(zip(gt,est)):
                D = cdist(estPts.T, [gtPt]) / pixels_per_cm
                dMean[pt,fr] = np.mean(D)
                dMin[pt,fr]  = min(D)
                dMax[pt,fr]  = max(D)

    showMax = 3
    bins = 10000
    binrange = [-1/bins, 500]

    for c,use in enumerate([dMin,dMean,dMax]):
        for p,pt in enumerate(use):
            counts,usedbins = np.histogram(pt,bins,range=binrange,density=True)
            ax[int(p/4),p%4].plot(usedbins[1:], counts.cumsum()/bins*binrange[1], colors[c], ls=dashes[mouse])
            ax[int(p/4),p%4].set_title(order[p])
            ax[int(p/4),p%4].set_xlim([0,showMax])
            ax[int(p/4),p%4].plot([1,1],[0,1],'k--')
            ax[int(p/4),p%4].plot([0,showMax],[.8,.8],'k--')

        counts,usedbins = np.histogram(use.ravel(),bins,range=binrange,density=True)
        ax[1,3].plot(usedbins[1:], counts.cumsum()/bins*binrange[1], colors[c], ls=dashes[mouse])
        ax[1,3].plot([1,1],[0,1],'k--')
        ax[1,3].plot([0,showMax],[.8,.8],'k--')
        ax[1,3].set_title('all');
        ax[1,3].set_xlim([0,showMax])

fig.savefig('s3://{}/performance.pdf'.format(lbl_bucket), bbox_inches='tight')